In [2]:
import os
import re
import random
from typing import List, Dict, Any, Optional
from collections import defaultdict
from time import time
from glob import glob
import numpy as np
import gym
from textworld import EnvInfos
import textworld.gym
import spacy
import substring

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_lg")

#import torch

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
gamefiles = []
GAMES_PATH = "./sample_games/"
for file in os.listdir(GAMES_PATH):
    if file.endswith('.ulx'):
        gamefiles.append(GAMES_PATH + file)

In [6]:
def play(agent, path, max_step=50, nb_episodes=10, verbose=True):
    request_infos = agent.select_additional_infos
    request_infos.max_score = True  # Needed to normalize the scores.

    gamefiles = [path]
    if os.path.isdir(path):
        gamefiles = glob(os.path.join(path, "*.ulx"))

    env_id = textworld.gym.register_games(gamefiles,
                                          request_infos=request_infos,
                                          max_episode_steps=max_step)
    env = gym.make(env_id)  # Create a Gym environment to play the text game.
    if verbose:
        if os.path.isdir(path):
            print(os.path.dirname(path), end="")
        else:
            print(os.path.basename(path), end="")

    # Collect some statistics: nb_steps, final reward.
    avg_moves, avg_scores, avg_norm_scores = [], [], []
    for no_episode in range(nb_episodes):
        obs, infos = env.reset()  # Start new episode.

        score = 0
        done = False
        nb_moves = 0
        while not done:
            command = agent.act(obs, score, done, infos)
            obs, score, done, infos = env.step(command)
            nb_moves += 1

        # Let the agent know the game is done.
        agent.act(obs, score, done, infos)

        if verbose:
            print(".", end="")
        avg_moves.append(nb_moves)
        avg_scores.append(score)
        avg_norm_scores.append(score / infos["max_score"])

    env.close()
    msg = "  \tavg. steps: {:5.1f}; avg. score: {:4.1f} / {}."
    if verbose:
        if os.path.isdir(path):
            print(msg.format(np.mean(avg_moves), np.mean(avg_norm_scores), 1))
        else:
            print(msg.format(np.mean(avg_moves), np.mean(
                avg_scores), infos["max_score"]))

In [7]:
class CustomAgent():
    
    def __init__(self) -> None:
        self._initialized = False
        self._epsiode_has_started = False
        self.mode = "test"
        self.command_history = []
        self.history_counter = 0
        
        # read in vocab.txt and map to id
        with open("./vocab.txt") as f:
            self.word_vocab = f.read().split("\n")
        self.word2id = {}
        self.id2word = []
        for i, w in enumerate(self.word_vocab):
            self.word2id[w] = i
            self.id2word.append(w)
        
        self.qvalues = [0] * len(self.id2word)

    def train(self) -> None:
        self.mode = "train"
        self.transitions = []
        self.last_score = 0
        self.no_train_step = 0
        self.moves = 0
        self.command_history = []
        self.history_counter = 0

    def test(self) -> None:
        self.mode = "test"

    @property
    def select_additional_infos(self) -> EnvInfos:
        """
        Returns what additional information should be made available at each game step.

        Requested information will be included within the `infos` dictionary
        passed to `CustomAgent.act()`. To request specific information, create a
        :py:class:`textworld.EnvInfos <textworld.envs.wrappers.filter.EnvInfos>`
        and set the appropriate attributes to `True`. The possible choices are:

        * `description`: text description of the current room, i.e. output of the `look` command;
        * `inventory`: text listing of the player's inventory, i.e. output of the `inventory` command;
        * `max_score`: maximum reachable score of the game;
        * `objective`: objective of the game described in text;
        * `entities`: names of all entities in the game;
        * `verbs`: verbs understood by the the game;
        * `command_templates`: templates for commands understood by the the game;
        * `admissible_commands`: all commands relevant to the current state;

        In addition to the standard information, game specific information
        can be requested by appending corresponding strings to the `extras`
        attribute. For this competition, the possible extras are:

        * `'recipe'`: description of the cookbook;
        * `'walkthrough'`: one possible solution to the game (not guaranteed to be optimal);

        Example:
            Here is an example of how to request information and retrieve it.

            >>> from textworld import EnvInfos
            >>> request_infos = EnvInfos(description=True, inventory=True, extras=["recipe"])
            ...
            >>> env = gym.make(env_id)
            >>> ob, infos = env.reset()
            >>> print(infos["description"])
            >>> print(infos["inventory"])
            >>> print(infos["extra.recipe"])

            Handicap is defined as follows
                max_score, has_won, has_lost,               # Handicap 0
                description, inventory, verbs, objective,   # Handicap 1
                command_templates,                          # Handicap 2
                entities,                                   # Handicap 3
                extras=["recipe"],                          # Handicap 4
                admissible_commands,                        # Handicap 5
        """
        
        return EnvInfos(description=True, inventory=False, verbs=True, extras=["recipe"], 
                        command_templates=True, max_score = True, admissible_commands=False, 
                        has_won=True, has_lost=True)
      

    def _init(self) -> None:
        """ Initialize the agent. """
        self._initialized = True

        # [You can insert code here.]
    
    def _get_word_id(self, word):
        """
        look up id of a word in dictionary & add word to dictionaries if unknown.
        """
        if word not in self.word2id:        
            self.id2word.append(word)
            self.qvalues.append(0)
            self.word2id[word] = len(self.word2id)
        return self.word2id[word]

    
    def _get_word_by_id(self, id):
        """
        look up word by id in dictionary.
        returns None, if id does not exist.
        """
        if id < len(self.id2word):
            return self.id2word[id]
        else:
            return None
    
    def _get_qvalue(self, id):
        """
        Getter for qvalues (admissible commands with scores).
        """
        if id < len(self.qvalues):
            return self.qvalues[id]
        else:
            return None
    
    def _tokenize(self, text):
        """
        Simple tokenizer: strip out all non-alphabetic characters.
        """
        text = re.sub("[^a-zA-Z0-9\- ]", " ", text)
        word_ids = list(map(self._get_word_id, text.split()))
        return word_ids
    
    def _start_episode(self, obs: List[str], infos: Dict[str, List[Any]]) -> None:
        """
        Prepare the agent for the upcoming episode.
        Arguments:
            obs: Initial feedback for each game.
            infos: Additional information for each game.
        """
        if not self._initialized:
            self._init()

        self._epsiode_has_started = True

        # [You can insert code here.]

    def _end_episode(self, obs: List[str], scores: List[int], infos: Dict[str, List[Any]]) -> None:
        """
        Tell the agent the episode has terminated.
        Arguments:
            obs: Previous command's feedback for each game.
            score: The score obtained so far for each game.
            infos: Additional information for each game.
        """
        self._epsiode_has_started = False

        # [You can insert code here.]
    
    def update_qvalues(self, command : List[int],reward):
        """
        Updates admissible commands with additionally rewards.
        """
        for id in command:
            self.qvalues[id] += reward
    
    def calc_avg_qvalue(self, command) -> int:
        """
        Gets the average of all qvalues as a sum.
        """
        sum = 0
        for id in command:
            sum += self._get_qvalue(id)
        sum = sum/ len(command)
        return sum

    def choose_best_cmd(self,commands) -> int:
        """
        Gets randomly one of the best rewarded admissible commands.
        """
        avg_qvalues =  []
        for cmd in commands:
            avg_qvalues.append(self.calc_avg_qvalue(cmd))
        max_value = max(avg_qvalues)
        max_commands = []
        for i in range(len(commands)):
            if(max_value == avg_qvalues[i]):
                max_commands.append(i)
        return random.choice(max_commands)
    
    def get_alternative_cmd(self, commands) -> int:
        """
        Idea of an alternative command, if the current is used in the last 3 commands.
        Chooses the second best command.
        """
        avg_qvalues =  []
        for cmd in commands:
            avg_qvalues.append(self.calc_avg_qvalue(cmd))
     
        first_max = max(avg_qvalues[0],avg_qvalues[1]) 
        second_max = min(avg_qvalues[0],avg_qvalues[1]) 

        for i in range(2,len(avg_qvalues)): 
            if avg_qvalues[i] > first_max: 
                second_max = first_max
                first_max=avg_qvalues[i] 
            else: 
                if avg_qvalues[i]>second_max: 
                    second_max=avg_qvalues[i] 
        return avg_qvalues.index(second_max)   
    
    def check_last_3_usages(self, cmds, infos, action):
        """
        building a historylist of commands to get out of a loop of death o_o
        """
        if self.command_history is not None:
            last_commands = self.command_history[-3:]
            
        # compare-counters for comparing actions from the 3 last moves.
        if self.history_counter > self.moves:
            same_command_in_list = True 
            counter_use_same_command = 0
            """ 
            If the current admissible command (action) is one of the last 3 commands, the while-loop
            will be skipped. 
            If the same random chosen command is going 2 times through the while-loop, after tahat, the loops
            will be skipped, too.
            """
            while same_command_in_list is True and counter_use_same_command < 3:
                try:
                    if last_commands.index(action):
                        best_index = self.get_alternative_cmd(cmds)
                        action = infos[best_index]
                        counter_use_same_command += 1
                    else:
                        same_command_in_list = False
                except ValueError:
                    same_command_in_list = False
                    print("last_commands List does not contain value")
                    
        # adds valid action to the command-history
        self.command_history.append(action)
        return last_commands

    def get_nouns_from_description(self, descr):
        """
        Get all nouns from description for finding admissible commands.
        """
        descr_nlp = nlp(descr)
        noun_list = []
        last_word_list = []
        sentences = descr.split(".")
        adm_objs = []
        
        # get the last word of a sentence from descr. because they can be significant.
        # reason: some words will not be found by nlp.
        # puts them in a list
        for sentence in sentences:
            last_word_list.append(sentence.split(" ")[-1]) 
            
        # get all nouns from the description with nlp. also not neccessary nouns.
        for chunk in descr_nlp.noun_chunks:
            if chunk is not None:
                noun_list.append(chunk.root.text)

        # compares last words of a list with all nouns in description, relevant words will be saved in a list
        for word in last_word_list:
            if word in noun_list:
                adm_objs.append(word)
        adm_objs.remove('TextWorld') # need better solution
        adm_objs.remove('it') # need better solution
        #print("adm_objs: ", adm_objs)
                    
        return adm_objs

    def get_nouns_from_recipe(self, recipe):
        """
        Get all nouns from recipe for finding admissible commands.
        """
        rec = recipe.replace("\n", " ")
        rec = rec.split(":")
        
        recipe_nlp = nlp(rec[2]) # get relevant nouns
        noun_list = []
        #for chunk in recipe_nlp.ent:
        for chunk in recipe_nlp.noun_chunks:
            if chunk is not None:
                noun_list.append(chunk.text)
        noun_list = noun_list[:-1] # drops meal 
        print("nouns from recipe:", noun_list)
        return noun_list
    
    def create_all_possible_commands(self, similiar_list, nouns_list):
        """
        Try to create possible commands. 
        Missing commands like '...into frigde...'
        open fridge, close fridge
        go east, go west, go north, go south
        take knife, drop knife
        put ... into ..., put ... on ..., take ... from ...
        """
        # without recipe but description, can be filtered out of the text. but this is another task
        commands = ['open fridge', 'close fridge', 'take knife', 'drop knife']
        #commands = []
        for verb in similiar_list:
            for noun in nouns_list:
                #cmd = verb.lemma_ + " " + noun
                cmd = verb + " " + noun
                if cmd not in commands:
                    commands.append(cmd)
        #print("created adm cmds: ", commands)
        return commands
    
    def create_adm_cmds(self, nouns_list, descr, poss_cmds):
        """
        Try to create admissible commands. 
        """
        adm_cmds = []
        for noun in nouns_list:
            if descr.find(noun) >= -1:
                for cmd in poss_cmds:
                    if noun in cmd:
                        adm_cmds.append(cmd)
        #print("adm_cmds: ", adm_cmds)
        return adm_cmds

    def act(self, obs: str, score: int, done: bool, 
            infos: Dict[str, List[Any]]) -> Optional[List[str]]:
        """
        Acts upon the current list of observations.

        One text command must be returned for each observation.

        Arguments:
            obs: Previous command's feedback for each game.
            scores: The score obtained so far for each game.
            dones: Whether a game is finished.
            infos: Additional information for each game.

        Returns:
            Text commands to be performed (one per observation).
            If episode had ended (e.g. `all(dones)`), the returned
            value is ignored.

        Notes:
            Commands returned for games marked as `done` have no effect.
            The states for finished games are simply copy over until all
            games are done.
        """
        #if all(dones):
        #   self._end_episode(obs, scores, infos)
        #    return  # Nothing to return.

        if not self._epsiode_has_started:
            self._start_episode(obs, infos)
        
        cmd_tmpl = infos["command_templates"]
        descr = infos["description"]
        verbs = infos["verbs"]
        stopverbs = ["examine", "look"] # drop examine and look from adm_cmds
        new_verbs = [word for word in verbs if word not in stopverbs]
        recipe = infos["extra.recipe"]
        
        print(cmd_tmpl)
       
        # creating adm cmds from recipe and description
        nouns_list = self.get_nouns_from_recipe(recipe)
        poss_cmds = self.create_all_possible_commands(new_verbs, nouns_list)
        adm_cmds = self.create_adm_cmds(nouns_list, descr, poss_cmds)
        
        # creating adm cmds only from descr, the results make me cry.
#         nouns_list = self.get_nouns_from_recipe(recipe)
#         adm_nouns = self.get_nouns_from_description(infos["description"])
#         nouns_list.extend(adm_nouns) # all important nouns together

        
#         poss_cmds = self.create_all_possible_commands(new_verbs, nouns_list)
#         adm_cmds = self.create_adm_cmds(nouns_list, descr, poss_cmds)
        
        #choose best command based on "qvalues"
        cmds = []
        for i in range(len(adm_cmds)):
            cmds.append(self._tokenize(adm_cmds[i]))
        best_index = self.choose_best_cmd(cmds)
        action = adm_cmds[best_index]
        
        # alternative: choose best command based on "qvalues"
#         cmds = []
#         for i in range(len(infos["admissible_commands"])):
#             cmds.append(self._tokenize(infos["admissible_commands"][i]))
#         best_index = self.choose_best_cmd(cmds)
#         action = infos["admissible_commands"][best_index]
        
        
        # last_commands for debug information and find new action if one is used the last 3 times.
        last_commands = self.check_last_3_usages(cmds, adm_cmds, action)
        
        if self.mode == "test":
            return action
        
        #train mode, counter update
        self.no_train_step += 1 
        reward = 0
        self.moves += 1
        self.history_counter += 1
        
        #calculate rewards
        if self.transitions:
            reward = score - self.last_score
            self.last_score = score
            if infos["has_won"]:
                reward += 10
            if infos["has_lost"]:
                reward -= 10
            if reward == 0:
                reward -= 1
            self.update_qvalues(self.transitions[-1],reward)

        # Debug output
        if(self.no_train_step % 1 == 0):
            print("")
            #print(infos["admissible_commands"])
            #print(infos["extra.recipe"]) # type string
            print("Last 3 admissible commands: ", last_commands)
            print("train step:" + str(self.no_train_step))
            print("last reward:" + str(reward))
            print("obs in act: " + obs)
            command = "last command: "
            if self.transitions:
                for id in self.transitions[-1]:
                    command += " " + self._get_word_by_id(id)
                print(command)
        
        #save last command
        self.transitions.append(cmds[best_index])
        
        if done:
            if(infos["has_won"]):
                print("-------- WON GAME ----------")
            else:
                print("-------- LOST GAME ----------")
            print(" ")
            self.last_score = 0
            self.moves = 0
            self.transitions = []
            self.command_history = []
            
        return action

In [8]:
agent = CustomAgent()
agent.train()
starttime = time()
for i in range(5):
    print("-------------------------------------")
    print("GAME: " + str(i+1))
    print("-------------------------------------")
    play(agent, gamefiles[5])
print("Trained in {:.2f} secs".format(time() - starttime))


-------------------------------------
GAME: 1
-------------------------------------
tw-cooking-recipe3+take3+cook+cut+drop-vNZGCrQVhoJdt7Nx.ulx['chop {f} with {o}', 'close {c}', 'close {d}', 'cook {f} with {oven}', 'cook {f} with {stove}', 'cook {f} with {toaster}', 'dice {f} with {o}', 'drink {f}', 'drop {o}', 'eat {f}', 'examine {o}', 'examine {t}', 'go east', 'go north', 'go south', 'go west', 'insert {o} into {c}', 'inventory', 'lock {c} with {k}', 'lock {d} with {k}', 'look', 'open {c}', 'open {d}', 'prepare meal', 'put {o} on {s}', 'slice {f} with {o}', 'take {o}', 'take {o} from {c}', 'take {o} from {s}', 'unlock {c} with {k}', 'unlock {d} with {k}']
nouns from recipe: ['the red apple', 'the red apple', 'the red onion', 'the yellow potato', 'the yellow potato']

Last 3 admissible commands:  []
train step:1
last reward:0
obs in act: 


                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
            

nouns from recipe: ['the red apple', 'the red apple', 'the red onion', 'the yellow potato', 'the yellow potato']

Last 3 admissible commands:  ['eat the yellow potato', 'lock the yellow potato', 'cook the red apple']
train step:9
last reward:-1
obs in act: (with the red apple)
You need to take the red apple first.


last command:  cook the red apple
['chop {f} with {o}', 'close {c}', 'close {d}', 'cook {f} with {oven}', 'cook {f} with {stove}', 'cook {f} with {toaster}', 'dice {f} with {o}', 'drink {f}', 'drop {o}', 'eat {f}', 'examine {o}', 'examine {t}', 'go east', 'go north', 'go south', 'go west', 'insert {o} into {c}', 'inventory', 'lock {c} with {k}', 'lock {d} with {k}', 'look', 'open {c}', 'open {d}', 'prepare meal', 'put {o} on {s}', 'slice {f} with {o}', 'take {o}', 'take {o} from {c}', 'take {o} from {s}', 'unlock {c} with {k}', 'unlock {d} with {k}']
nouns from recipe: ['the red apple', 'the red apple', 'the red onion', 'the yellow potato', 'the yellow potato']

Last 3 admi

nouns from recipe: ['the red apple', 'the red apple', 'the red onion', 'the yellow potato', 'the yellow potato']

Last 3 admissible commands:  ['go the red apple', 'put the yellow potato', 'close the yellow potato']
train step:28
last reward:-1
obs in act: It isn't something you can close.


last command:  close the yellow potato
['chop {f} with {o}', 'close {c}', 'close {d}', 'cook {f} with {oven}', 'cook {f} with {stove}', 'cook {f} with {toaster}', 'dice {f} with {o}', 'drink {f}', 'drop {o}', 'eat {f}', 'examine {o}', 'examine {t}', 'go east', 'go north', 'go south', 'go west', 'insert {o} into {c}', 'inventory', 'lock {c} with {k}', 'lock {d} with {k}', 'look', 'open {c}', 'open {d}', 'prepare meal', 'put {o} on {s}', 'slice {f} with {o}', 'take {o}', 'take {o} from {c}', 'take {o} from {s}', 'unlock {c} with {k}', 'unlock {d} with {k}']
nouns from recipe: ['the red apple', 'the red apple', 'the red onion', 'the yellow potato', 'the yellow potato']

Last 3 admissible commands:  ['

['chop {f} with {o}', 'close {c}', 'close {d}', 'cook {f} with {oven}', 'cook {f} with {stove}', 'cook {f} with {toaster}', 'dice {f} with {o}', 'drink {f}', 'drop {o}', 'eat {f}', 'examine {o}', 'examine {t}', 'go east', 'go north', 'go south', 'go west', 'insert {o} into {c}', 'inventory', 'lock {c} with {k}', 'lock {d} with {k}', 'look', 'open {c}', 'open {d}', 'prepare meal', 'put {o} on {s}', 'slice {f} with {o}', 'take {o}', 'take {o} from {c}', 'take {o} from {s}', 'unlock {c} with {k}', 'unlock {d} with {k}']
nouns from recipe: ['the red apple', 'the red apple', 'the red onion', 'the yellow potato', 'the yellow potato']

Last 3 admissible commands:  ['prepare the red apple', 'insert the red onion', 'insert the yellow potato']
train step:38
last reward:-1
obs in act: (in the yellow potato)
You need to take the yellow potato first.


last command:  insert the yellow potato
['chop {f} with {o}', 'close {c}', 'close {d}', 'cook {f} with {oven}', 'cook {f} with {stove}', 'cook {f} w

['chop {f} with {o}', 'close {c}', 'close {d}', 'cook {f} with {oven}', 'cook {f} with {stove}', 'cook {f} with {toaster}', 'dice {f} with {o}', 'drink {f}', 'drop {o}', 'eat {f}', 'examine {o}', 'examine {t}', 'go east', 'go north', 'go south', 'go west', 'insert {o} into {c}', 'inventory', 'lock {c} with {k}', 'lock {d} with {k}', 'look', 'open {c}', 'open {d}', 'prepare meal', 'put {o} on {s}', 'slice {f} with {o}', 'take {o}', 'take {o} from {c}', 'take {o} from {s}', 'unlock {c} with {k}', 'unlock {d} with {k}']
nouns from recipe: ['the red apple', 'the red apple', 'the red onion', 'the yellow potato', 'the yellow potato']

Last 3 admissible commands:  ['take the red apple', 'take the red apple', 'slice the red onion']
train step:49
last reward:-1
obs in act: (with the red onion)
Cutting something requires a knife.


last command:  slice the red onion
['chop {f} with {o}', 'close {c}', 'close {d}', 'cook {f} with {oven}', 'cook {f} with {stove}', 'cook {f} with {toaster}', 'dice {

...

.['chop {f} with {o}', 'close {c}', 'close {d}', 'cook {f} with {oven}', 'cook {f} with {stove}', 'cook {f} with {toaster}', 'dice {f} with {o}', 'drink {f}', 'drop {o}', 'eat {f}', 'examine {o}', 'examine {t}', 'go east', 'go north', 'go south', 'go west', 'insert {o} into {c}', 'inventory', 'lock {c} with {k}', 'lock {d} with {k}', 'look', 'open {c}', 'open {d}', 'prepare meal', 'put {o} on {s}', 'slice {f} with {o}', 'take {o}', 'take {o} from {c}', 'take {o} from {s}', 'unlock {c} with {k}', 'unlock {d} with {k}']
nouns from recipe: ['the red apple', 'the red apple', 'the red onion', 'the yellow potato', 'the yellow potato']
last_commands List does not contain value

Last 3 admissible commands:  []
train step:2327
last reward:0
obs in act: 


                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
                      | $$   | $$__     \$

['chop {f} with {o}', 'close {c}', 'close {d}', 'cook {f} with {oven}', 'cook {f} with {stove}', 'cook {f} with {toaster}', 'dice {f} with {o}', 'drink {f}', 'drop {o}', 'eat {f}', 'examine {o}', 'examine {t}', 'go east', 'go north', 'go south', 'go west', 'insert {o} into {c}', 'inventory', 'lock {c} with {k}', 'lock {d} with {k}', 'look', 'open {c}', 'open {d}', 'prepare meal', 'put {o} on {s}', 'slice {f} with {o}', 'take {o}', 'take {o} from {c}', 'take {o} from {s}', 'unlock {c} with {k}', 'unlock {d} with {k}']
nouns from recipe: ['the red apple', 'the red apple', 'the red onion', 'the yellow potato', 'the yellow potato']
last_commands List does not contain value

Last 3 admissible commands:  ['go the red apple', 'inventory the yellow potato', 'open the yellow potato']
train step:2338
last reward:-1
obs in act: It isn't something you can open.


last command:  open the yellow potato
['chop {f} with {o}', 'close {c}', 'close {d}', 'cook {f} with {oven}', 'cook {f} with {stove}', '

nouns from recipe: ['the red apple', 'the red apple', 'the red onion', 'the yellow potato', 'the yellow potato']
last_commands List does not contain value

Last 3 admissible commands:  ['open the red apple', 'drink the red onion', 'prepare the red onion']
train step:2348
last reward:-1
obs in act: I didn't understand that sentence.

last command:  prepare the red onion
['chop {f} with {o}', 'close {c}', 'close {d}', 'cook {f} with {oven}', 'cook {f} with {stove}', 'cook {f} with {toaster}', 'dice {f} with {o}', 'drink {f}', 'drop {o}', 'eat {f}', 'examine {o}', 'examine {t}', 'go east', 'go north', 'go south', 'go west', 'insert {o} into {c}', 'inventory', 'lock {c} with {k}', 'lock {d} with {k}', 'look', 'open {c}', 'open {d}', 'prepare meal', 'put {o} on {s}', 'slice {f} with {o}', 'take {o}', 'take {o} from {c}', 'take {o} from {s}', 'unlock {c} with {k}', 'unlock {d} with {k}']
nouns from recipe: ['the red apple', 'the red apple', 'the red onion', 'the yellow potato', 'the yellow p

nouns from recipe: ['the red apple', 'the red apple', 'the red onion', 'the yellow potato', 'the yellow potato']
last_commands List does not contain value

Last 3 admissible commands:  ['chop the yellow potato', 'inventory the yellow potato', 'chop the red apple']
train step:2360
last reward:-1
obs in act: I only understood you as far as wanting to take inventory.

last command:  inventory the yellow potato
['chop {f} with {o}', 'close {c}', 'close {d}', 'cook {f} with {oven}', 'cook {f} with {stove}', 'cook {f} with {toaster}', 'dice {f} with {o}', 'drink {f}', 'drop {o}', 'eat {f}', 'examine {o}', 'examine {t}', 'go east', 'go north', 'go south', 'go west', 'insert {o} into {c}', 'inventory', 'lock {c} with {k}', 'lock {d} with {k}', 'look', 'open {c}', 'open {d}', 'prepare meal', 'put {o} on {s}', 'slice {f} with {o}', 'take {o}', 'take {o} from {c}', 'take {o} from {s}', 'unlock {c} with {k}', 'unlock {d} with {k}']
nouns from recipe: ['the red apple', 'the red apple', 'the red oni

nouns from recipe: ['the red apple', 'the red apple', 'the red onion', 'the yellow potato', 'the yellow potato']

Last 3 admissible commands:  ['take the red apple', 'insert the red apple', 'take the red apple']
train step:2370
last reward:-1
obs in act: You already have that.



last command:  take the red apple
['chop {f} with {o}', 'close {c}', 'close {d}', 'cook {f} with {oven}', 'cook {f} with {stove}', 'cook {f} with {toaster}', 'dice {f} with {o}', 'drink {f}', 'drop {o}', 'eat {f}', 'examine {o}', 'examine {t}', 'go east', 'go north', 'go south', 'go west', 'insert {o} into {c}', 'inventory', 'lock {c} with {k}', 'lock {d} with {k}', 'look', 'open {c}', 'open {d}', 'prepare meal', 'put {o} on {s}', 'slice {f} with {o}', 'take {o}', 'take {o} from {c}', 'take {o} from {s}', 'unlock {c} with {k}', 'unlock {d} with {k}']
nouns from recipe: ['the red apple', 'the red apple', 'the red onion', 'the yellow potato', 'the yellow potato']
last_commands List does not contain value

Last 3

In [19]:
agent.test()
play(agent, gamefiles[5])

tw-cooking-recipe3+take3+cook+cut+drop-vNZGCrQVhoJdt7Nx.ulx['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'pu

['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'inser

['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare'

['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'inser

['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'inser

['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare'

['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'inser

['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'inser

['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
.['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare

In [64]:
for i in range(len(agent.qvalues)):
    if(agent.qvalues[i] != 0 ):
        print(agent._get_word_by_id(i) + " : " + str(agent.qvalues[i]))

apple : -656
chop : -120
close : -120
cook : -120
dice : -121
drink : -120
drop : -121
eat : -127
examine : -120
go : -120
insert : -121
inventory : -120
lock : -120
look : -121
onion : -656
open : -120
potato : -983
prepare : -120
put : -121
red : -1312
slice : -121
take : -122
the : -2295
unlock : -120
yellow : -983


In [20]:
agent.train()
starttime = time()
for i in range(5):
    print("-------------------------------------")
    print("GAME: " + str(i+1))
    print("-------------------------------------")
    play(agent, gamefiles[8])
print("Trained in {:.2f} secs".format(time() - starttime))

-------------------------------------
GAME: 1
-------------------------------------
tw-cooking-recipe3+cook+cut+drop+go12-B10qS58OTbe0T8Qk.ulx['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']

Last 3 admissible commands:  []
train step:1
last reward:0
obs in act: 


                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
                      | $$   | $$__     \$$\/  $$   | $$          
                      | $$   | $$  \     >$$  $$    | $$          
                      | $$   | $$$$$    /  $$$$\    | $$          
                      | $$   | $$_____ |  $$ \$$\   | $$          
                      | $$   | $$     \| $$  | $$   | $$          
                       \$$    \$$$$$$$$ \$$   \$$    \$$          
              __       __   ____


Last 3 admissible commands:  ['drop the red hot pepper', 'take the red hot pepper', 'cook the red hot pepper']
train step:27
last reward:-1
obs in act: (with the red hot pepper)
You need to take the red hot pepper first.


last command:  cook the red hot pepper
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']

Last 3 admissible commands:  ['take the red hot pepper', 'cook the red hot pepper', 'drink the red hot pepper']
train step:28
last reward:-1
obs in act: You need to take the red hot pepper first.


last command:  drink the red hot pepper
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']

Last 3 admissible commands:  ['cook the red hot pepper', 'drink the red hot pepper', 'unlock the red hot pepper']
train step:29
last reward:-1
obs in act: (with the red hot pepper)
You need to take the


Last 3 admissible commands:  ['chop the red hot pepper', 'chop the red hot pepper', 'drink the red hot pepper']
train step:50
last reward:-1
obs in act: You need to take the red hot pepper first.


last command:  drink the red hot pepper
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']

Last 3 admissible commands:  ['chop the red hot pepper', 'drink the red hot pepper', 'inventory the red hot pepper']
train step:51
last reward:-1
obs in act: I only understood you as far as wanting to take inventory.

last command:  inventory the red hot pepper
-------- LOST GAME ----------
 
.['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value

Last 3 admissible commands:  []
train step:52
last reward:0
obs in act: 


                    ________  ________  __    __  _

['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value

Last 3 admissible commands:  ['slice the red hot pepper', 'inventory the red hot pepper', 'drop the red hot pepper']
train step:68
last reward:-1
obs in act: I only understood you as far as wanting to take inventory.

last command:  inventory the red hot pepper
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']

Last 3 admissible commands:  ['inventory the red hot pepper', 'drop the red hot pepper', 'chop the red hot pepper']
train step:69
last reward:-1
obs in act: You drop the red hot pepper on the ground.


last command:  drop the red hot pepper
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_comm

['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value

Last 3 admissible commands:  ['dice the red hot pepper', 'chop the red hot pepper', 'chop the red hot pepper']
train step:91
last reward:-1
obs in act: (with the red hot pepper)
Cutting something requires a knife.


last command:  chop the red hot pepper
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value

Last 3 admissible commands:  ['chop the red hot pepper', 'chop the red hot pepper', 'take the red hot pepper']
train step:92
last reward:-1
obs in act: You're carrying too many things already.



last command:  take the red hot pepper
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice'

...

last_commands List does not contain value

Last 3 admissible commands:  ['slice the red hot pepper', 'close the red hot pepper', 'go the yellow potato']
train step:2023
last reward:-1
obs in act: That's not something you can enter.


last command:  go the yellow potato
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value

Last 3 admissible commands:  ['close the red hot pepper', 'go the yellow potato', 'inventory the yellow potato']
train step:2024
last reward:-1
obs in act: I only understood you as far as wanting to take inventory.

last command:  inventory the yellow potato
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value

Last 3 admissible commands:  ['go the yellow potato', 'inventory the yellow potato', 'cho

['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value

Last 3 admissible commands:  ['close the yellow potato', 'inventory the red hot pepper', 'drink the red hot pepper']
train step:2040
last reward:-1
obs in act: You cannot drink the red hot pepper.


last command:  drink the red hot pepper
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value

Last 3 admissible commands:  ['inventory the red hot pepper', 'drink the red hot pepper', 'insert the red hot pepper']
train step:2041
last reward:-1
obs in act: (in the red hot pepper)
You can't put something inside itself.


last command:  insert the red hot pepper
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare

last_commands List does not contain value

Last 3 admissible commands:  ['close the red hot pepper', 'cook the red hot pepper', 'dice the red hot pepper']
train step:2065
last reward:-1
obs in act: (with the red hot pepper)
Cutting something requires a knife.


last command:  dice the red hot pepper
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value

Last 3 admissible commands:  ['cook the red hot pepper', 'dice the red hot pepper', 'prepare the yellow potato']
train step:2066
last reward:-1
obs in act: I didn't understand that sentence.

last command:  prepare the yellow potato
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value

Last 3 admissible commands:  ['dice the red hot pepper', 'prepare the yellow potato'

['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value

Last 3 admissible commands:  ['lock the yellow potato', 'drink the yellow potato', 'cook the yellow potato']
train step:2091
last reward:-1
obs in act: What do you want to cook the yellow potato with?

last command:  cook the yellow potato
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']

Last 3 admissible commands:  ['drink the yellow potato', 'cook the yellow potato', 'open the red hot pepper']
train step:2092
last reward:-1
obs in act: It isn't something you can open.


last command:  open the red hot pepper
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value

La

['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value

Last 3 admissible commands:  ['go the yellow potato', 'chop the yellow potato', 'lock the yellow potato']
train step:2115
last reward:-1
obs in act: (with the yellow potato)
You need to take the yellow potato first.


last command:  lock the yellow potato
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
last_commands List does not contain value

Last 3 admissible commands:  ['chop the yellow potato', 'lock the yellow potato', 'slice the red hot pepper']
train step:2116
last reward:-1
obs in act: (with the yellow potato)
Cutting something requires a knife.


last command:  slice the red hot pepper
['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'go', 'insert', 'inventory', 'lock', 'open', 'pr

In [44]:
# versuche
wordids = agent._tokenize("cook a meal")
print(wordids)

print(agent._get_word_id("cook"))
print(agent._get_word_by_id(4664))

[4664, 785, 11449]
4664
cook


In [78]:
def play_render(agent,gamefile):
    requested_infos = agent.select_additional_infos
    env_id = textworld.gym.register_games([gamefile], requested_infos)

    env = gym.make(env_id)
    obs, infos = env.reset()

    env.render()  # Print the initial observation.

    score = 0
    done = False
    while not done:
        command = agent.act(obs,score,done,infos)
        ob, score, done, infos = env.step(command)
        env.render()

In [79]:
agent.test()
play_render(agent, gamefiles[5])




                    ________  ________  __    __  ________
                   |        \|        \|  \  |  \|        \
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$
                      | $$   | $$__     \$$\/  $$   | $$
                      | $$   | $$  \     >$$  $$    | $$
                      | $$   | $$$$$    /  $$$$\    | $$
                      | $$   | $$_____ |  $$ \$$\   | $$
                      | $$   | $$     \| $$  | $$   | $$
                       \$$    \$$$$$$$$ \$$   \$$    \$$
              __       __   ______   _______   __        _______
             |  \  _  |  \ /      \ |       \ |  \      |       \
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$  | $$
             | $$ $$\$$\$$| $$  | $$| $$$$$$$\| $$      | $$  | $$
             | $$$$  \$$$$| $$__/ $$| $$  | $$| $$_____ | $$__/ $$
          